In [2]:
import pandas as pd
import numpy as np
from pyreadr import read_r
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

In [3]:
import matplotlib.pyplot as plt

In [4]:
data = read_r('data/train.rds')

In [5]:
df = data[None]

In [6]:
df.head(3)

,SampleID,CellType,IGLV3-19,IGHV4-34,IGKC,IGHA1,IGLC3,S100A2,SCGB3A1,IGHG1,...,AC092794.2,AC016722.3,LINC01119,LINC01291,AC010132.4,C7orf69,AL672277.1,FAM66D,AC007298.2,AL139022.2
rownames,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGATATACG_2,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGCGTGAAC_2,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGCTACCTA_2,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df.describe()

,SampleID,CellType,IGLV3-19,IGHV4-34,IGKC,IGHA1,IGLC3,S100A2,SCGB3A1,IGHG1,...,AC092794.2,AC016722.3,LINC01119,LINC01291,AC010132.4,C7orf69,AL672277.1,FAM66D,AC007298.2,AL139022.2
count,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,...,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000
mean,6.333398,2.808810,0.935829,0.090065,1.156428,0.368514,0.072652,0.585728,1.428880,0.735481,...,0.000322,0.000484,0.000290,0.000419,0.000387,0.000516,0.000226,0.000451,0.000258,0.000290
std,3.120921,1.882997,58.852290,8.994020,24.451987,14.369674,4.153750,14.337457,34.247756,17.644792,...,0.017955,0.021988,0.017034,0.020471,0.019668,0.022709,0.015023,0.021243,0.016060,0.017034
min,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,9.000000,10.000000,6547.000000,1278.000000,1368.000000,1062.000000,482.000000,986.000000,2076.000000,1071.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
df['CellType'] = df['CellType'].astype('int8')

1. There are a lot of genes without Expression at all. We can discard them.
#There is no need to check for cells (rows) since each has a label so there should be some gene expression (I actually checked it and no changes in shape)

Also, We can further take some risk by removing features with low variance. Let's set 0.01

In [8]:
#1 Remove all with low variance (ignore SampleID and CellType)
selected_df = df.iloc[:, 2:]

selector = VarianceThreshold(threshold= .01)

reduced_data = selector.fit_transform(selected_df)

scaler = StandardScaler()

reduced_data = scaler.fit_transform(reduced_data)

In [9]:
selected_columns = selected_df.columns[selector.get_support()] #extract the remaining column names

clean_data = pd.DataFrame(reduced_data, columns=selected_columns) #All genes after dimensionality reduction

In [10]:
final_data = pd.concat( [df.iloc[:, :2].reset_index(drop=True), clean_data.reset_index(drop=True)] , axis=1) #reset_index because they have different index values

In [10]:
final_data.describe()

,SampleID,CellType,IGLV3-19,IGHV4-34,IGKC,IGHA1,IGLC3,S100A2,SCGB3A1,IGHG1,...,BLM,STARD4,CCDC171,ITSN1,PPM1H,AHR,HPS5,MEI1,PNMA1,MAP3K9
count,31011.000000,31011.000000,3.101100e+04,31011.000000,3.101100e+04,3.101100e+04,3.101100e+04,3.101100e+04,3.101100e+04,31011.000000,...,3.101100e+04,3.101100e+04,31011.000000,3.101100e+04,3.101100e+04,3.101100e+04,3.101100e+04,3.101100e+04,3.101100e+04,3.101100e+04
mean,6.333398,2.808810,-3.666017e-18,0.000000,7.332033e-18,3.666017e-18,6.415529e-18,-3.666017e-18,-2.016309e-17,0.000000,...,-6.048927e-17,-7.515334e-17,0.000000,-5.499025e-17,9.165041e-19,-8.798440e-17,4.399220e-17,-1.466407e-17,1.035650e-16,-4.811647e-17
std,3.120921,1.882997,1.000016e+00,1.000016,1.000016e+00,1.000016e+00,1.000016e+00,1.000016e+00,1.000016e+00,1.000016,...,1.000016e+00,1.000016e+00,1.000016,1.000016e+00,1.000016e+00,1.000016e+00,1.000016e+00,1.000016e+00,1.000016e+00,1.000016e+00
min,2.000000,1.000000,-1.590158e-02,-0.010014,-4.729460e-02,-2.564570e-02,-1.749090e-02,-4.085362e-02,-4.172253e-02,-0.041683,...,-1.725802e-01,-3.538894e-01,-0.103406,-2.458198e-01,-8.821074e-02,-3.866566e-01,-4.277646e-01,-1.902331e-01,-2.602697e-01,-7.083841e-02
25%,2.000000,2.000000,-1.590158e-02,-0.010014,-4.729460e-02,-2.564570e-02,-1.749090e-02,-4.085362e-02,-4.172253e-02,-0.041683,...,-1.725802e-01,-3.538894e-01,-0.103406,-2.458198e-01,-8.821074e-02,-3.866566e-01,-4.277646e-01,-1.902331e-01,-2.602697e-01,-7.083841e-02
50%,9.000000,2.000000,-1.590158e-02,-0.010014,-4.729460e-02,-2.564570e-02,-1.749090e-02,-4.085362e-02,-4.172253e-02,-0.041683,...,-1.725802e-01,-3.538894e-01,-0.103406,-2.458198e-01,-8.821074e-02,-3.866566e-01,-4.277646e-01,-1.902331e-01,-2.602697e-01,-7.083841e-02
75%,9.000000,2.000000,-1.590158e-02,-0.010014,-4.729460e-02,-2.564570e-02,-1.749090e-02,-4.085362e-02,-4.172253e-02,-0.041683,...,-1.725802e-01,-3.538894e-01,-0.103406,-2.458198e-01,-8.821074e-02,-3.866566e-01,-4.277646e-01,-1.902331e-01,-2.602697e-01,-7.083841e-02
max,9.000000,10.000000,1.112305e+02,142.086684,5.589998e+01,7.388119e+01,1.160241e+02,6.873117e+01,6.057636e+01,60.657091,...,3.582665e+01,1.272875e+01,80.219472,1.706633e+01,4.278801e+01,1.896984e+01,1.892770e+01,3.632549e+01,4.779955e+01,4.569520e+01


In [18]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31011 entries, 0 to 31010
Columns: 4010 entries, SampleID to MAP3K9
dtypes: float64(4009), int8(1)
memory usage: 948.5 MB


In [ ]:
print(final_data.dtypes.value_counts())

float64    4009
int8          1
Name: count, dtype: int64


2. For each Cell type, how many samples do we have?

In [19]:
final_data.groupby(by='CellType')['CellType'].count()

CellType
1       449
2     24766
3       830
4       389
5        65
6        21
7      3700
8       559
9        66
10      166
Name: CellType, dtype: int64

In [20]:
final_data.groupby(by='SampleID')['SampleID'].count()

SampleID
2.0     8153
3.0     2566
6.0     3409
9.0    16883
Name: SampleID, dtype: int64

3. Save cleaned data

In [21]:
final_data.to_csv('data/cleaned_data.csv', index=False)

In [23]:
x = pd.read_csv('data/cleaned_data.csv')

In [25]:
x.describe()

,SampleID,CellType,IGLV3-19,IGHV4-34,IGKC,IGHA1,IGLC3,S100A2,SCGB3A1,IGHG1,...,BLM,STARD4,CCDC171,ITSN1,PPM1H,AHR,HPS5,MEI1,PNMA1,MAP3K9
count,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,...,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000,31011.000000
mean,6.333398,2.808810,0.000003,-0.000001,-0.000009,-0.000004,0.000004,-0.000007,0.000005,-0.000005,...,0.000002,-0.000119,0.000006,-0.000083,-0.000006,0.000021,0.000028,0.000056,-0.000008,-0.000006
std,3.120921,1.882997,1.000073,1.000250,0.999979,0.999994,0.999888,1.000087,1.000020,0.999976,...,1.000141,1.000072,1.000095,0.999886,0.999845,1.000223,1.000052,1.000057,0.999980,1.000130
min,2.000000,1.000000,-0.015900,-0.010017,-0.047302,-0.025650,-0.017487,-0.040863,-0.041718,-0.041687,...,-0.172607,-0.354004,-0.103394,-0.245850,-0.088196,-0.386719,-0.427734,-0.190186,-0.260254,-0.070862
25%,2.000000,2.000000,-0.015900,-0.010017,-0.047302,-0.025650,-0.017487,-0.040863,-0.041718,-0.041687,...,-0.172607,-0.354004,-0.103394,-0.245850,-0.088196,-0.386719,-0.427734,-0.190186,-0.260254,-0.070862
50%,9.000000,2.000000,-0.015900,-0.010017,-0.047302,-0.025650,-0.017487,-0.040863,-0.041718,-0.041687,...,-0.172607,-0.354004,-0.103394,-0.245850,-0.088196,-0.386719,-0.427734,-0.190186,-0.260254,-0.070862
75%,9.000000,2.000000,-0.015900,-0.010017,-0.047302,-0.025650,-0.017487,-0.040863,-0.041718,-0.041687,...,-0.172607,-0.354004,-0.103394,-0.245850,-0.088196,-0.386719,-0.427734,-0.190186,-0.260254,-0.070862
max,9.000000,10.000000,111.250000,142.125000,55.906250,73.875000,116.000000,68.750000,60.562500,60.656250,...,35.812500,12.726562,80.250000,17.062500,42.781250,18.968750,18.921875,36.312500,47.812500,45.687500


In [26]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31011 entries, 0 to 31010
Columns: 4010 entries, SampleID to MAP3K9
dtypes: float64(4009), int64(1)
memory usage: 948.7 MB
